In [ ]:
import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfds


In [ ]:
mnist_dataset,mnist_info=tfds.load(name='mnist',with_info=True,as_supervised=True)

# Preprocessing

In [ ]:
#train test split
mnist_train,mnist_test=mnist_dataset['train'],mnist_dataset['test']
mnist_train


In [ ]:
#validation set
num_validation_samples=0.1*mnist_info.splits['train'].num_examples
num_validation_samples=tf.cast(num_validation_samples,tf.int64)

num_test_samples=mnist_info.splits['test'].num_examples
num_test_samples=tf.cast(num_test_samples,tf.int64)


In [ ]:
mnist_info.splits['train'].num_examples

In [ ]:
num_test_samples

In [ ]:
#scaling mnist matrix data from 0-255 to 0-1
def scale(image,label):
    image=tf.cast(image,tf.float32)
    image=image/255.
    return image,label

scaled_train_and_validation_data=mnist_train.map(scale)


In [ ]:
scaled_train_and_validation_data

In [ ]:
#scale test data
test_data=mnist_test.map(scale)
test_data

In [ ]:
#shuffle data before spliting
buffer_size=10000
shuffled_train_and_validation_data=scaled_train_and_validation_data.shuffle(buffer_size)

In [ ]:
#split train and validation datasets
validation_data=shuffled_train_and_validation_data.take(num_validation_samples)
train_data=shuffled_train_and_validation_data.skip(num_validation_samples)

In [ ]:
BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)

validation_data = validation_data.batch(num_validation_samples)

test_data = test_data.batch(num_test_samples)


validation_inputs, validation_targets = next(iter(validation_data))
test_data


# Outline the model

In [ ]:
input_size = 784
output_size = 10

hidden_layer_size = 50

model = tf.keras.Sequential([
    

    tf.keras.layers.Flatten(input_shape=(28, 28, 1)), # input layer

    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 1st hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 2nd hidden layer
    tf.keras.layers.Dense(output_size, activation='softmax') # output layer
])

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

no_of_epochs=5
model.fit(train_data,epochs=no_of_epochs,validation_data=(validation_inputs,validation_targets),verbose=2)

In [ ]:
model.save('mnist_digit.h5')

In [ ]:
model.evaluate(test_data)

In [ ]:
test_loss, test_accuracy = model.evaluate(test_data)
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

In [ ]:
from keras.models import load_model
model=load_model('mnist_digit.h5')


In [ ]:
res=model.predict(test_data)

In [ ]:
for i in res.round():
    print(list(map(int,i.round())).index(1))